In [24]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset

from typing import Union
import turbo
from rul_dataframe import RUL_DataFrame
import copy
import numpy as np

In [2]:
tr, te, rl = turbo.getTFDataset(set = 4)
turbo.addTFlinear(tr, te, rl)
tr_df = rul_dataframe.RUL_DataFrame(df = tr, label_cols = ['linear'])
te_df = rul_dataframe.RUL_DataFrame(df = te, label_cols = ['linear'])

In [3]:
rul_df = copy.deepcopy(tr_df)
rul_df_test = copy.deepcopy(te_df)

In [4]:
class TorchRULIterable():

    def __init__(self, rul_df, label, start = 1) -> None:
        self.current_id = start
        self.last_id = rul_df.df[rul_df.id_col].iloc[-1]
        self.label = label
        self.rul_df = rul_df

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_id <= self.last_id:
            x = self.rul_df.df.loc[self.rul_df.df[self.rul_df.id_col] == self.current_id, self.rul_df.data_cols]
            y = self.rul_df.df.loc[self.rul_df.df[self.rul_df.id_col] == self.current_id, self.label]
            self.current_id += 1
            return torch.tensor(x.values.astype(np.float32)), torch.tensor(y.values.astype(np.float32))
        else:
            raise StopIteration

In [56]:
tri = TorchRULIterable(rul_df = rul_df, label = 'linear')

In [67]:
x,y = next(tri)

In [20]:
len(test)

2

In [108]:
class TorchRULDataset(Dataset):

    def __init__(self, rul_df: RUL_DataFrame, label: str, sequence_length: Union[None, int] = None):
        self.rul_df = rul_df
        self.label = label
        self.seq_l = sequence_length
        self.ids = rul_df.df[rul_df.id_col].unique()

        self.items = []
        for i in self.ids:
            trajectory = rul_df.df.loc[rul_df.df[rul_df.id_col] == i].index
            start = trajectory[0]
            end = trajectory[-1]
            self.items.append([i, start, end])

        self.id_col = rul_df.id_col
        self.len = len(self.ids)
        if self.seq_l:
            '''
            if fixed sequence length
            '''

            self.len = 0
            self.items = []
            current_item = 0

            for i in self.ids:
                
                trajectory = rul_df.df.loc[rul_df.df[rul_df.id_col] == i].index
                leni = len(trajectory)

                if leni >= self.seq_l:
                    # calculate how many trajectories with seq_l fit in id
                    n = leni - self.seq_l + 1
                    self.len += n
                    #go through trajectories
                    for j in range(n):
                        start = trajectory[j]
                        end = trajectory[j + self.seq_l - 1]
                        current_item += 1
                        self.items.append([current_item, start, end])
                else:
                   raise Exception('selected seqence legth too long')

    def __getitem__(self, item):
        trajectory = self.rul_df.df.iloc[self.items[item - 1][1]:(self.items[item - 1][2] + 1)]
        if self.items[item - 1][0] == item:
            trajectory = self.rul_df.df.iloc[self.items[item - 1][1]:(self.items[item - 1][2] + 1)]
        else:
            for i in range(len(self.items)):
                if self.items[i][0] == item:
                     trajectory = self.rul_df.df.iloc[self.items[i][1]:(self.items[i][2] + 1)]
        # Transform it to Tensor
        x = torch.tensor(trajectory[self.rul_df.data_cols].values)
        y = torch.tensor(trajectory[self.label].values)
        return x, y

    def __len__(self):
        return self.len

In [109]:
tdf = TorchRULDataset(rul_df = rul_df, label = 'linear', sequence_length = 30)

In [110]:
tdf_loader = DataLoader(tdf, batch_size=2)

In [111]:
train_features, train_labels = next(iter(tdf_loader))

In [112]:
train_labels

tensor([[ 29,  28,  27,  26,  25,  24,  23,  22,  21,  20,  19,  18,  17,  16,
          15,  14,  13,  12,  11,  10,   9,   8,   7,   6,   5,   4,   3,   2,
           1,   0],
        [320, 319, 318, 317, 316, 315, 314, 313, 312, 311, 310, 309, 308, 307,
         306, 305, 304, 303, 302, 301, 300, 299, 298, 297, 296, 295, 294, 293,
         292, 291]])

In [82]:
rul_df.df[rul_df.df['id'] == 5]

,id,dt,set1,set2,set3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,linear
1201,5,1,10.0014,0.2500,100.0,489.05,605.16,1495.18,1313.85,10.52,...,2388.17,8117.05,8.6631,0.03,369,2319,100.00,28.52,17.1688,192
1202,5,2,35.0021,0.8403,100.0,449.44,556.01,1362.30,1130.14,5.48,...,2387.87,8050.68,9.3478,0.02,333,2223,100.00,14.75,8.9903,191
1203,5,3,42.0061,0.8400,100.0,445.00,549.76,1354.68,1121.03,3.91,...,2387.65,8074.99,9.3953,0.02,330,2212,100.00,10.63,6.3188,190
1204,5,4,24.9991,0.6200,60.0,462.54,537.19,1265.86,1041.95,7.05,...,2027.98,7863.62,10.9364,0.02,307,1915,84.93,14.16,8.4967,189
1205,5,5,24.9991,0.6200,60.0,462.54,536.95,1269.30,1043.15,7.05,...,2028.08,7863.91,10.9334,0.02,308,1915,84.93,14.36,8.5399,188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,5,189,42.0064,0.8400,100.0,445.00,550.48,1368.79,1145.17,3.91,...,2389.13,8150.76,9.4508,0.02,333,2212,100.00,10.52,6.3349,4
1390,5,190,25.0011,0.6200,60.0,462.54,537.10,1268.80,1061.82,7.05,...,2029.30,7922.33,11.0187,0.02,310,1915,84.93,14.06,8.4474,3
1391,5,191,35.0022,0.8411,100.0,449.44,556.57,1380.88,1141.91,5.48,...,2389.39,8131.67,9.4143,0.02,336,2223,100.00,14.79,8.9248,2
1392,5,192,35.0066,0.8417,100.0,449.44,557.24,1378.72,1143.91,5.48,...,2389.46,8127.58,9.3749,0.02,338,2223,100.00,14.65,8.7800,1


In [102]:
tdf.len

54028

In [84]:
rul_df.df['id'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [114]:
for batch_idx, data in enumerate(tdf_loader):
    print('batch {}, x shape {}, y shape {}'.format(batch_idx, data[0].shape, data[1].shape))

batch 0, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 1, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 2, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 3, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 4, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 5, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 6, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 7, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 8, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 9, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 10, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 11, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 12, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 13, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 14, x shap

batch 178, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 179, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 180, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 181, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 182, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 183, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 184, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 185, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 186, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 187, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 188, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 189, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 190, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 191, x shape torch.Size([2, 30, 21]), y shape torch.Size([

batch 361, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 362, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 363, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 364, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 365, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 366, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 367, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 368, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 369, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 370, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 371, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 372, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 373, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 374, x shape torch.Size([2, 30, 21]), y shape torch.Size([

batch 553, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 554, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 555, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 556, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 557, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 558, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 559, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 560, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 561, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 562, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 563, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 564, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 565, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 566, x shape torch.Size([2, 30, 21]), y shape torch.Size([

batch 729, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 730, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 731, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 732, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 733, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 734, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 735, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 736, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 737, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 738, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 739, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 740, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 741, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 742, x shape torch.Size([2, 30, 21]), y shape torch.Size([

...

batch 26093, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26094, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26095, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26096, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26097, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26098, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26099, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26100, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26101, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26102, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26103, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26104, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26105, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26106, x shape torch.Size([2, 30

batch 26290, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26291, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26292, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26293, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26294, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26295, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26296, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26297, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26298, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26299, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26300, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26301, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26302, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26303, x shape torch.Size([2, 30

batch 26488, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26489, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26490, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26491, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26492, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26493, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26494, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26495, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26496, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26497, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26498, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26499, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26500, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26501, x shape torch.Size([2, 30

batch 26686, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26687, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26688, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26689, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26690, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26691, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26692, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26693, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26694, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26695, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26696, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26697, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26698, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26699, x shape torch.Size([2, 30

batch 26885, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26886, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26887, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26888, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26889, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26890, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26891, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26892, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26893, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26894, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26895, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26896, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26897, x shape torch.Size([2, 30, 21]), y shape torch.Size([2, 30])
batch 26898, x shape torch.Size([2, 30